## The Quantum Approximate Optimisation Algorithm (QAOA) using TKET.

Seyon Sivarajah
seyon.sivarajah@quantinuum.com

## The Max-Cut problem

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
G = nx.Graph()
G.add_edges_from([(0,1), (1,2), (2,0)])
plt.figure(figsize=(2,2))
nx.draw(G, node_color=['red', 'blue', 'red'])
plt.show()

There are \\(2^3\\) possible assignments of colour to nodes. In general there are \\( 2^n \\). The Max-cut problem can then be stated as that of finding the colour assignment which maximises the number of edges between vertices of a different colour.

## Quantum Approximate Optimization Algorithm (QAOA)

Introduced in 'A Quantum Approximate Optimization Algorithm' (found at https://arxiv.org/abs/1411.4028). The idea is to prepare a quantum state which encodes a solution to the Max-cut problem.


This is a variational algorithm, which is to say that a paramaterised state is prepared, with the parameters varied to improve the solution. We will have $2p$ parameters where p is our number of layers. In particular, the state prepared has the form 



\\[ \left| \psi \left( \beta, \gamma \right) \right\rangle = U \left( \beta_m \right) U \left( \gamma_m \right) ... U \left( \beta_0 \right) U \left( \gamma_0 \right) \left| \psi_0 \right\rangle \\]
where
\\[ U \left( \beta_i \right) = e^{i \beta H_B} \quad \& \quad U \left( \gamma_i \right) = e^{i \gamma H_P} \\]
with \\( H_B \\) and \\( H_P \\) depending on the problem instance. 

## Cost function for Maxcut
$
\begin{equation}
C= \sum_{(i,j)} x_i(1-x_j)
\end{equation}
$

For the previous 3 vertex graph the *problem Hamiltonian* is
\\[ H_P = \frac{1}{2} \big[ \left( Z \otimes Z \otimes I \right) + \left( Z \otimes I \otimes Z \right) + \left( I \otimes Z \otimes Z \right) \big] \\]


where you will notice that there is a \\( Z \otimes Z \\) acting between each vertex which is connected by an edge.

The *mixer Hamiltonian* has the form 
\\[ H_B =   \left( X \otimes I \otimes I \right) + \left( I \otimes X \otimes I \right) + \left( I \otimes I \otimes X \right)  \\]


where you will notice that there is an \\( X \\) acting on each vertex.

## Cost function for Maxcut

A solution to maxcut can be found by maximising the following cost function $C$ .



$
\begin{equation}
C= \sum_{(i,j)} x_i(1-x_j)
\end{equation}
$

Here $x_i$ and $x_j$ are the the "colours" of each vertex.  

$x_i,x_j \in \{0,1\}$

$x_i(1-x_j)=0$ if $x_i=x_j$ and $ x_i(1-x_j)=1$ if the terms are not equal.

We want to encode our Maxcut cost function as a Hamiltonain. To do this we can perform the following translation.

$
\begin{equation}
x_i \mapsto \frac{1}{2}(1-Z_i)
\end{equation}
$


The Pauli Z operator can be used to distinguish between the $|0\rangle$ and $|1\rangle$ basis states as these are eigenstates with eigenvalues $\pm 1$ .

$
\begin{equation}
H_P = \frac{1}{2}\sum_{<jk>} (-Z_j \,Z_k +I )
\end{equation}
$


$
\begin{equation}
H_B = \sum_i X_i
\end{equation}
$

Here we use the the convention that $X_i$ means a Pauli X operator will be applied to the "ith" qubit and the identity operator will be applied to all other qubits in the circuit.

In [ ]:
import warnings
warnings.filterwarnings("ignore")

# Circuit Construction for QAOA

In [ ]:
import networkx as nx
max_cut_graph_edges = [(0,1), (1,2), (1,3), (3,4), (4,5), (4,6)]
n_nodes = 7

max_cut_graph = nx.Graph()
max_cut_graph.add_edges_from(max_cut_graph_edges)
nx.draw(max_cut_graph, labels={node: node for node in max_cut_graph.nodes()})

expected_results = [(0,1,0,0,1,0,0), (1,0,1,1,0,1,1)]

<span style="color:red">**EXERCISE 1**</span>
What are the max cut solutions for the graph with edges:


`[(0, 2), (0, 3), (1, 2), (1, 3), (2,4), (2, 5), (3, 4), (3, 5)]`

## Define Cost Hamiltonian: $\gamma H$

$
\begin{equation}
H_P = \frac{1}{2}\sum_{<jk>} (-Z_j \,Z_k +I )
\end{equation}
$

In [ ]:
from typing import List, Tuple, Any
from pytket.utils import QubitPauliOperator
from pytket.pauli import QubitPauliString, Pauli
from pytket import Qubit

def qaoa_graph_to_cost_hamiltonian(edges: List[Tuple[int, int]], cost_angle: float) -> QubitPauliOperator:
    qpo_dict = {QubitPauliString(): len(edges)*0.5*cost_angle}
    for e in edges:
        term_string = QubitPauliString([Qubit(e[0]), Qubit(e[1])], [Pauli.Z, Pauli.Z])
        qpo_dict[term_string] = -0.5*cost_angle
    return QubitPauliOperator(qpo_dict)

cost_angle = 1.0
cost_ham_qpo = qaoa_graph_to_cost_hamiltonian(max_cut_graph_edges, cost_angle)
print(cost_ham_qpo)

$
\begin{equation}
H_P = 3 I^{\otimes 6} -0.5 \big[ Z_0 Z_1 + Z_1 Z_2 +Z_1 Z_3 +Z_3 Z_4 +Z_4 Z_5 +Z_4 Z_6  \big]
\end{equation}
$

Using the same index convention as above

<span style="color:red">**EXERCISE 2**</span>
What is the cost hamiltonian for the graph in Exercise 1? Write it down, then verify it using the function above.

## Hamiltonian Circuit

In [ ]:
from pytket.utils import gen_term_sequence_circuit
from pytket import Circuit
from pytket.circuit import display

cost_ham_circuit = gen_term_sequence_circuit(cost_ham_qpo, Circuit(n_nodes))
display.render_circuit_jupyter(cost_ham_circuit)

In [ ]:
from pytket.transform import Transform

Transform.DecomposeBoxes().apply(cost_ham_circuit)
display.render_circuit_jupyter(cost_ham_circuit)

## Construction of the Mixer Hamiltonian: $\beta B$

In [ ]:
mixer_angle = 0.8
mixer_ham_qpo =  QubitPauliOperator({QubitPauliString([Qubit(i)], [Pauli.X]): mixer_angle for i in range(n_nodes)})
mixer_ham_circuit = gen_term_sequence_circuit(mixer_ham_qpo, Circuit(n_nodes))
Transform.DecomposeBoxes().apply(mixer_ham_circuit)
display.render_circuit_jupyter(mixer_ham_circuit)

## Define the Initial State

In [ ]:
def qaoa_initial_circuit(n_qubits: int) -> Circuit:
    c = Circuit(n_qubits)
    for i in range(n_qubits):
        c.H(i)
    return c

superposition_circuit = qaoa_initial_circuit(n_nodes)

display.render_circuit_jupyter(superposition_circuit)

## Construct QAOA Circuit

In [ ]:
def qaoa_max_cut_circuit(edges: List[Tuple[int, int]],
                         n_nodes: int,
                         mixer_angles: List[float],
                         cost_angles: List[float]) -> Circuit:
    
    assert len(mixer_angles) == len(cost_angles)
    
    # initial state
    qaoa_circuit = qaoa_initial_circuit(n_nodes)
    
    # add cost and mixer terms to state
    for cost, mixer in zip(cost_angles, mixer_angles):
        cost_ham = qaoa_graph_to_cost_hamiltonian(edges, cost)
        mixer_ham = QubitPauliOperator({QubitPauliString([Qubit(i)], [Pauli.X]): mixer for i in range(n_nodes)})
        qaoa_circuit.append(gen_term_sequence_circuit(cost_ham, Circuit(n_nodes)))
        qaoa_circuit.append(gen_term_sequence_circuit(mixer_ham, Circuit(n_nodes)))
        
    Transform.DecomposeBoxes().apply(qaoa_circuit)
    return qaoa_circuit

## Measurement to the Circuit

In [ ]:
from pytket.circuit import Bit

def qaoa_gen_all_measurement_circuit(ham: QubitPauliOperator,
                                     my_prep_circuit: Circuit):

    all_measurement_circuit = []

    for qps, cof in ham._dict.items():    
        measurement_circuit = Circuit()
        measured_qbs = []
        for qb, p in qps.map.items():
            if p == Pauli.I:
                continue
            measured_qbs.append(qb)
            measurement_circuit.add_qubit(qb)
            if p == Pauli.X: # unused in our case
                measurement_circuit.H(qb)
            elif p == Pauli.Y: # unused in our case
                measurement_circuit.Rx(0.5, qb)
        bits = []
        for b_idx, qb in enumerate(measured_qbs):
            unit = Bit(b_idx)
            bits.append(unit)
            measurement_circuit.add_bit(unit, False)
            measurement_circuit.Measure(qb, unit)

        c = my_prep_circuit.copy()
        c.append(measurement_circuit)
        all_measurement_circuit.append((c, (qps, bits)))    
    
    return all_measurement_circuit

## Calculate the Energy of the Results

In [ ]:
from pytket.backends.backendresult import BackendResult

def qaoa_estimated_energy(results: BackendResult,
                          all_measurement_circuit: List[Tuple[Circuit, Tuple[Any, int]]],
                          ham: QubitPauliOperator):

    organised_results = {}

    for r, c in zip(results, all_measurement_circuit):
        bits=c[1][1]
        counts = r.get_counts(cbits=bits)
        expectations = expectation_from_counts(counts)
        # c[1][0] is the qubit pauli string
        organised_results[c[1][0]] = expectations
    
    # calculate expectation of hamiltonian observable:
    estimated_energy = 0

    for qps, coef in ham._dict.items():
        expect = organised_results[qps]
        estimate = expect * coef
        estimated_energy += estimate
    
    return estimated_energy

## Set up QAOA Instance

In [ ]:
from pytket.backends import Backend
from typing import Callable
import numpy as np
from pytket.utils import expectation_from_counts

def qaoa_instance(backend: Backend,
                  compiler_pass: Callable[[Circuit], bool],
                  guess_mixer_angles: np.array,
                  guess_cost_angles: np.array,
                  shots: int = 5000) -> float:

    assert len(guess_mixer_angles) == len(guess_cost_angles)
    
    # step 1: get state guess
    my_prep_circuit = qaoa_max_cut_circuit(max_cut_graph_edges,
                                           n_nodes,
                                           guess_mixer_angles,
                                           guess_cost_angles)
    
    # step 2: get problem hamiltonian
    ham = qaoa_graph_to_cost_hamiltonian(max_cut_graph_edges, 1.0)
    
    # step 3: prepare measurement circuit
    all_measurement_circuit = qaoa_gen_all_measurement_circuit(ham, my_prep_circuit)
        
    # step 4: compile the circuit for the backend
    for c in all_measurement_circuit:
        compiler_pass(c[0])

    # step 5: run measurement circuit on backend   
    handles = backend.process_circuits([c[0] for c in all_measurement_circuit], shots)
    
    # step 6: get expectations
    results = backend.get_results(handles)
    estimated_energy = qaoa_estimated_energy(results, all_measurement_circuit, ham)
        
    return estimated_energy

### Simplified qaoa instane

In [ ]:
from typing import List, Tuple
from pytket.backends.backendresult import BackendResult


def max_cut_energy(edges: List[Tuple[int, int]], results: BackendResult) -> float:
    energy = 0.0
    dist = results.get_distribution()
    for i, j in edges:
        energy += sum((meas[i] ^ meas[j]) * prob for meas, prob in dist.items())

    return energy


def qaoa_instance_simple(
    backend: Backend,
    compiler_pass: Callable[[Circuit], bool],
    guess_mixer_angles: np.array,
    guess_cost_angles: np.array,
    seed: int,
    shots: int = 5000,
) -> float:
    # step 1: get state guess
    my_prep_circuit = qaoa_max_cut_circuit(
        max_cut_graph_edges, n_nodes, guess_mixer_angles, guess_cost_angles
    )
    measured_circ = my_prep_circuit.copy().measure_all()
    compiler_pass(measured_circ)
    res = backend.run_circuit(measured_circ, shots, seed=seed)

    return max_cut_energy(max_cut_graph_edges, res)

## Optimise Energy by Guessing Parameters

In [ ]:
def qaoa_optimise_energy(compiler_pass: Callable[[Circuit], bool],
                         backend: Backend,
                         iterations: int = 100,
                         n: int = 3,
                         shots: int = 5000,
                         seed: int= 12345):
    
    highest_energy = 0    
    best_guess_mixer_angles = [0 for i in range(n)]    
    best_guess_cost_angles = [0 for i in range(n)]
    rng = np.random.default_rng(seed)
    # guess some angles (iterations)-times and try if they are better than the best angles found before
    
    for i in range(iterations):
        
        guess_mixer_angles = rng.uniform(0, 1, n)
        guess_cost_angles = rng.uniform(0, 1, n)
        
        qaoa_energy = qaoa_instance_simple(backend,
                                    compiler_pass,
                                    guess_mixer_angles,
                                    guess_cost_angles,
                                    seed=seed,
                                    shots=shots)
        
        if(qaoa_energy > highest_energy):
            
            print("new highest energy found: ", qaoa_energy)
            
            best_guess_mixer_angles = guess_mixer_angles
            best_guess_cost_angles = guess_cost_angles
            highest_energy = qaoa_energy
            
    print("highest energy: ", highest_energy)
    print("best guess mixer angles: ", best_guess_mixer_angles)
    print("best guess cost angles: ", best_guess_cost_angles)
    return best_guess_mixer_angles, best_guess_cost_angles

<span style="color:red">**EXERCISE 3**</span>
Can you define an alternative `qaoa_optimise_energy` which uses a classical optimiser to find the best angles? (Hint: try playing with `scipy.optimize.minimize`)


## Calculate the State for the final Parameters

In [ ]:
import qaoaplotfunctions as qaoaplot

def qaoa_calculate(backend: Backend,
                   compiler_pass: Callable[[Circuit], bool],
                   shots: int = 5000,
                   iterations: int = 100,
                   seed: int = 12345,
                  ) -> float:
    
    # find the parameters for the highest energy
    best_mixer, best_cost = qaoa_optimise_energy(compiler_pass,
                                                 backend,
                                                 iterations,
                                                 3,
                                                 shots=shots,
                                                 seed=seed)
    
    # get the circuit with the final parameters of the optimisation:
    my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges,
                                           n_nodes,
                                           best_mixer,
                                           best_cost)

    my_qaoa_circuit.measure_all()

    compiler_pass(my_qaoa_circuit)
    handle = backend.process_circuit(my_qaoa_circuit, shots, seed=seed)

    result = backend.get_result(handle)    

    qaoaplot.plotresults(result, expected_results)


## Results with the Noiseless Simulator

In [ ]:
from pytket.extensions.qiskit import AerBackend

backend = AerBackend()
comp = backend.get_compiled_circuit

In [ ]:
%%time
qaoa_calculate(backend, backend.default_compilation_pass(1).apply, shots = 5000, iterations = 100, seed=12345)

## Results with the Noisy Simulator

In [ ]:
# This requires an IBMQ account, set yours as appropriate here
from pytket.extensions.qiskit.backends.config import set_ibmq_config 
# set the below as appropriate
# set_ibmq_config(ibmq_api_token="",
#                 hub="", 
#                 group="", 
#                 project="")


In [ ]:
from pytket.extensions.qiskit import IBMQEmulatorBackend

backend = IBMQEmulatorBackend('ibmq_guadalupe')


In [ ]:
qaoa_calculate(backend, backend.default_compilation_pass(1).apply, shots = 5000, iterations = 100, seed=12347)

## Circuit of one QAOA Instance with one Mixer and one Cost Angle

In [ ]:
my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])
display.render_circuit_jupyter(my_qaoa_circuit)

In [ ]:
from pytket.circuit import OpType

print("Number of gates in Circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in Circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

<span style="color:red">**EXERCISE 4**</span>
Adding more angles generally leads to a more accurate solution, but on the other hand adds more layers and therefore more noise. For a given noisy simulator backend, find the best balance between number of angles and noise. (**Hint**: what are you trying to optimise? How can you measure this quantity?)

## Compile Circuit for Device

In [ ]:
guadalupe_backend = IBMQEmulatorBackend('ibmq_guadalupe')
my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])
compiled_qaoa_circuit=guadalupe_backend.get_compiled_circuit(my_qaoa_circuit)
print("Number of gates in circuit: ", compiled_qaoa_circuit.n_gates)
print("Number of CX gates in circuit: ", compiled_qaoa_circuit.n_gates_of_type(OpType.CX))
display.render_circuit_jupyter(compiled_qaoa_circuit)

In [ ]:
from pytket.passes import OptimisePhaseGadgets

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])
OptimisePhaseGadgets().apply(my_qaoa_circuit)
print("Number of gates in Circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in Circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))
display.render_circuit_jupyter(my_qaoa_circuit)

In [ ]:
from pytket.placement import NoiseAwarePlacement

guadalupe_info = guadalupe_backend.backend_info
guadalupe_placer = NoiseAwarePlacement(arc=guadalupe_info.architecture,
                                       node_errors=guadalupe_info.averaged_node_gate_errors,
                                       link_errors=guadalupe_info.averaged_edge_gate_errors,
                                       readout_errors=guadalupe_info.averaged_readout_errors)

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])

print(guadalupe_placer.get_placement_map(my_qaoa_circuit))
guadalupe_placer.modify_config(depth_limit = 100, max_matches = 20000)
print(guadalupe_placer.get_placement_map(my_qaoa_circuit))

In [ ]:
from pytket.passes import CXMappingPass

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])

guadalupe_mapping_pass = CXMappingPass(
    guadalupe_info.architecture,
    guadalupe_placer,
    directed_cx=False
)
guadalupe_mapping_pass.apply(my_qaoa_circuit)
print("Number of gates in Circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in Circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

display.render_circuit_jupyter(my_qaoa_circuit)

# Predicates

In [ ]:
from pytket.predicates import GateSetPredicate, NoMidMeasurePredicate, ConnectivityPredicate

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])

ibm_gateset = GateSetPredicate({OpType.X, OpType.SX, OpType.Rz, OpType.CX, OpType.Measure})

print("check ibm gateset predicate: ", ibm_gateset.verify(my_qaoa_circuit))

midmeasure = NoMidMeasurePredicate()

print("check mid measure predicate: ", midmeasure.verify(my_qaoa_circuit))

ibm_conn_predicate = ConnectivityPredicate(guadalupe_info.architecture)

print("check ibm connectivity predicate predicate: ", ibm_conn_predicate.verify(my_qaoa_circuit))

In [ ]:
from pytket.passes.auto_rebase import auto_rebase_pass
ibm_rebase = auto_rebase_pass({OpType.CX, OpType.X, OpType.SX, OpType.Rz, OpType.CX})
ibm_rebase.apply(my_qaoa_circuit)

print("check ibm gateset predicate: ", ibm_gateset.verify(my_qaoa_circuit))

print("Number of gates in Circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in Circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

## Local Gate Optimisation

In [ ]:
from pytket.passes import FullPeepholeOptimise

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])


FullPeepholeOptimise().apply(my_qaoa_circuit)
print("Number of gates in Circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in Circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

display.render_circuit_jupyter(my_qaoa_circuit)

## Sequence Pass

In [ ]:
from pytket.passes import SequencePass, RemoveRedundancies

sequence_pass = SequencePass([OptimisePhaseGadgets(),
                              FullPeepholeOptimise(),
                              guadalupe_mapping_pass,
                              FullPeepholeOptimise(allow_swaps=False),
                              ibm_rebase, RemoveRedundancies()])

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])
sequence_pass.apply(my_qaoa_circuit)
print("Number of gates in circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

display.render_circuit_jupyter(my_qaoa_circuit)

<span style="color:red">**EXERCISE 5**</span>

Define your own `SequencePass` from pytket passes ([docs](https://cqcl.github.io/tket/pytket/api/passes.html), [example](https://github.com/CQCL/pytket/blob/main/examples/compilation_example.ipynb)), such that it outperforms the one defined above in terms of number of gates or number of CX gates.

Verify your pass produces correct outputs for the ibmq_guadalupe backend.

## Compilation Units

In [ ]:
from pytket.predicates import CompilationUnit

my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges, n_nodes, [0.8], [1.0])


cu = CompilationUnit(my_qaoa_circuit, [ibm_gateset, midmeasure, ibm_conn_predicate])

print(cu.check_all_predicates())

sequence_pass.apply(cu)

print(cu.check_all_predicates())

print("Number of gates in circuit: ", my_qaoa_circuit.n_gates)
print("Number of CX gates in circuit: ", my_qaoa_circuit.n_gates_of_type(OpType.CX))

In [ ]:
backend = IBMQEmulatorBackend('ibmq_guadalupe')
comp = sequence_pass.apply
qaoa_calculate(backend, comp, shots = 5000, iterations = 100)

## Use symbolic compilation to reduce the compile time

In [ ]:
from sympy import symbols
from pytket.predicates import NoSymbolsPredicate

a, b = symbols("a b")
circ_symbolic = Circuit(2)
circ_symbolic.Ry(a, 0)
circ_symbolic.Ry(a, 1)
circ_symbolic.CX(0, 1)
circ_symbolic.Rz(b, 1)
circ_symbolic.CX(0, 1)

backend = AerBackend()

# compile once
circ_symbolic = backend.get_compiled_circuit(circ_symbolic)   

print(NoSymbolsPredicate().verify(circ_symbolic))

circ_runable = circ_symbolic.copy()
circ_runable.symbol_substitution({a : 0.1, b : 0.2})
    
print(NoSymbolsPredicate().verify(circ_runable))

# no need to compile again
handle = backend.process_circuit(circ_runable, 5000)

## Define Symbolic Parameters using Sympy

In [ ]:
a_s, b_s, c_s, d_s, e_s, f_s = symbols("a b c d e f")

In [ ]:
def qaoa_sym_all_measurement_circuit(compiler_pass: Callable[[Circuit], bool]) -> List[Tuple[Circuit, Tuple[Any, int]]]:

    # define the circuit
    my_prep_curcuit = qaoa_max_cut_circuit(max_cut_graph_edges,
                                               n_nodes,
                                               [a_s, b_s, c_s],
                                               [d_s, e_s, f_s])
    
    # get hamiltonian
    ham = qaoa_graph_to_cost_hamiltonian(max_cut_graph_edges, 1.0)
    
    # prepare meturement circuit
    all_measurement_circuit = []

    for qps, cof in ham._dict.items():
        measurement_circuit = Circuit()
        measured_qbs = []
        for qb, p in qps.map.items():
            if p == Pauli.I:
                continue
            measured_qbs.append(qb)
            measurement_circuit.add_qubit(qb)
            if p == Pauli.X:
                measurement_circuit.H(qb)
            elif p == Pauli.Y:
                measurement_circuit.Rx(0.5, qb)
        bits = []
        for b_idx, qb in enumerate(measured_qbs):
            unit = Bit(b_idx)
            bits.append(unit)
            measurement_circuit.add_bit(unit, False)
            measurement_circuit.Measure(qb, unit)

        c = my_prep_curcuit.copy()
        c.append(measurement_circuit)
        all_measurement_circuit.append([c, (qps, bits)])
        
    # compile circuit for backend
    for c in all_measurement_circuit:
        compiler_pass(c[0])
    
    return all_measurement_circuit

In [ ]:
def qaoa_sym_instance(backend: Backend,
                      all_measurement_circuit: List[Tuple[Circuit, Tuple[Any, int]]],
                      guess_mixer_angles: np.array,
                      guess_cost_angles: np.array,
                      shots: int = 5000) -> float:

    assert len(guess_mixer_angles) == len(guess_cost_angles)
    assert len(guess_mixer_angles) == 3
    
    ham = qaoa_graph_to_cost_hamiltonian(max_cut_graph_edges, 1.0)
    
    all_measurement_circuit_subs = []
                                                    
    for ci in all_measurement_circuit:
        circ = ci[0].copy()
        circ.symbol_substitution({a_s : guess_mixer_angles[0],
                                  b_s : guess_mixer_angles[1],
                                  c_s : guess_mixer_angles[2],
                                  d_s : guess_cost_angles[0],
                                  e_s : guess_cost_angles[1],
                                  f_s : guess_cost_angles[2]})
        all_measurement_circuit_subs.append((circ, ci[1]))

    handles = backend.process_circuits([c[0] for c in all_measurement_circuit_subs], shots)
    
    # get expectations
    results = backend.get_results(handles)

    organised_results = {}

    for r, c in zip(results, all_measurement_circuit_subs):
        bits=c[1][1]
        counts = r.get_counts(cbits=bits)
        expectations = expectation_from_counts(counts)
        organised_results[c[1][0]] = expectations
    
    # calculate expectation of hamiltonian observable
    estimated_energy = 0
    for qps, coef in ham._dict.items():
        expect = organised_results[qps]
        estimate = expect * coef
        estimated_energy += estimate    
        
    return estimated_energy

In [ ]:
def qaoa_sym_optimise_energy(backend: Backend,
                             compiler_pass: Callable[[Circuit], bool],
                             shots: int = 5000,
                             iterations: int = 100) -> Tuple[List[float], List[float]]:
    # find the parameters for the highest energy
    highest_energy = 0
    n = 3

    best_guess_mixer_angles = [0 for i in range(n)]
    best_guess_cost_angles = [0 for i in range(n)]

    # guess some angles (iterations)-times and try if they are better than the best angles found before    
    all_measurement_circuit = qaoa_sym_all_measurement_circuit(compiler_pass)    
    
    for i in range(iterations):
        
        guess_mixer_angles = np.random.uniform(0, 1, n)
        guess_cost_angles = np.random.uniform(0, 1, n)
        
        qaoa_energy = qaoa_sym_instance(backend,
                                        all_measurement_circuit,
                                        guess_mixer_angles,
                                        guess_cost_angles,
                                        shots)
        
        if(qaoa_energy > highest_energy):
            print("new highest energy found: ", qaoa_energy)
            best_guess_mixer_angles = guess_mixer_angles
            best_guess_cost_angles = guess_cost_angles
            highest_energy = qaoa_energy
            
    print("highest energy: ", highest_energy)
    print("best guess mixer angles: ", best_guess_mixer_angles)
    print("best guess cost angles: ", best_guess_cost_angles)
    
    return best_guess_mixer_angles, best_guess_cost_angles

In [ ]:
def qaoa_sym_calculate(backend: Backend,
                       compiler_pass: Callable[[Circuit], bool],
                       shots: int = 5000,
                       iterations: int = 100) -> float:
    
    best_mixer, best_cost = qaoa_sym_optimise_energy(backend, compiler_pass, shots, iterations)
    
    # calculate the result of the optimisation:
    my_qaoa_circuit = qaoa_max_cut_circuit(max_cut_graph_edges,
                                           n_nodes,
                                           best_mixer,
                                           best_cost)
    my_qaoa_circuit.measure_all()
    compiler_pass(my_qaoa_circuit)    
    handle = backend.process_circuit(my_qaoa_circuit, shots)
    result = backend.get_result(handle)
    qaoaplot.plotresults(result, expected_results)

In [ ]:
backend = IBMQEmulatorBackend('ibmq_guadalupe')
comp = sequence_pass.apply
qaoa_sym_calculate(backend, comp, shots = 5000, iterations = 100)

<span style="color:red">**EXERCISE 6**</span>

Measure the difference in time taken for compilation for the symbolic and non-symbolic instances.

## The Quantum Approximate Optimisation Algorithm (QAOA) using TKET.

### What have we covered?

## 1) Circuit Construction 
Helper methods for generating quantum circuits from Hamiltonians <br>
Defining functions for generating circuits to some specification <br>

## 2) Circuit Compilation 
Using Transform and Optimisation Passes to improve Circuit Compilation <br>
Changing configuration of passes i.e. NoiseAwarePlacement configuration <br>
Combining Passes into a Sequence Pass <br>
Symbolic compilation <br>